In [ ]:
print("ok")

In [1]:
%pwd

'/home/tarun/Desktop/AI Medical Chatbot/End-to-End-Medical-chatbot/research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd


'/home/tarun/Desktop/AI Medical Chatbot/End-to-End-Medical-chatbot'

In [7]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [8]:
# Extract Data from the PDF File

def load_pdf_file(data):
    loader= DirectoryLoader(data, 
                            glob="*.pdf",
                            loader_cls=PyPDFLoader
                            )
    
    documents=loader.load()

    return documents


In [9]:
extracted_data = load_pdf_file(data='Data/')

In [10]:
# # Print first 5 documents with their content
# print(f"Total number of documents: {len(extracted_data)}\n")
# print("First 5 documents:")
# for i, doc in enumerate(extracted_data[:5]):
#     print(f"\nDocument {i+1}:")
#     print(f"Source: {doc.metadata.get('source', 'No source')}")
#     print(f"Page: {doc.metadata.get('page', 'No page number')}")
#     print(f"Content preview: {doc.page_content[:200]}...")

In [11]:
#Split the data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 40000


In [13]:
# text_chunks

In [32]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [30]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    )
    return embeddings

# embeddings = download_hugging_face_embeddings()


In [31]:
embeddings = download_hugging_face_embeddings()

In [33]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
# query_result

Length 384


## PINECONE VECTOR DB SETUP

In [115]:
from dotenv import load_dotenv
load_dotenv()

True

In [116]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
print(PINECONE_API_KEY)

pcsk_6XRjRR_5jA2brukgbF1AHfqr2YBekphi6Dd1Jen9ygWYMGCMULFFiK2ubNpc29UZAe9s8v


In [113]:
from pinecone import Pinecone, ServerlessSpec
import os

# Make sure your API key is available
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

# Create index only if it doesn't already exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Must match your embeddings (MiniLM = 384)
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Connect to the index
index = pc.Index(index_name)

print("✅ Pinecone index ready:", index_name)


✅ Pinecone index ready: medicalbot


In [ ]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [ ]:
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [75]:
# Load Existing index
# Embed each chunk and upsert the embeddings into your Pinecone index.

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [77]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\nturnover (death and replacement) of skin cells.\nAcne vulgaris affecting a woman’s face. Acne is 

In [ ]:

OPEN_API_KEY = os.getenv("OPEN_API_KEY")

In [144]:
from langchain_openai import OpenAI
llm = OpenAI(api_key=OPEN_API_KEY ,temperature=0.4, max_tokens=500)

In [146]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     api_key=  OPEN_API_KEY
# )

# print(llm.invoke("Hello!").content)


In [136]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Pinecone as PineconeVectorStore


# Your LLM
llm = ChatOpenAI(api_key=OPEN_API_KEY, model="gpt-4o-mini", temperature=0.4)

# Prompt Template
system_prompt = (
    "You are an assistant for medical question-answering.\n"
    "Use the context below to answer accurately.\n"
    "If the answer is not in the context, reply: 'I don't know'.\n"
    "Keep the answer short and clear.\n\n"
    "Context:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}")
    ]
)



In [147]:
question = "Actually, from the last night, I'm getting belly pain, help me to know what is the problem?"
docs = retriever.invoke(question)
docs



[Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 411.0, 'page_label': '382', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 4505.0}, page_content='pain beginning around or above the navel. The pain,\nwhich may be severe or only achy and uncomfortable,\neventually moves into the right lower corner of the\nabdomen. There, it becomes more steady and more\nsevere, and often increases with movement, coughing,\nand so forth. The abdomen often becomes rigid and\ntender to the touch. Increasing rigidity and tenderness\nindicates an increased likelihood of perforation and\nperitonitis.\nLoss of appetite is very common. Nausea and'),
 Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1592.0, 'page_label': '1563', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Dat

In [148]:
context = "\n\n".join(doc.page_content for doc in docs)
response = llm.invoke(prompt.format(context=context, question=question))

In [149]:
print(response)

content="I don't know." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 409, 'total_tokens': 413, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CWG3fRmQpgUzyE0ghv2a3joY6Tfg1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--9ade1575-3e78-4fe1-8a9f-9ce9bc53e837-0' usage_metadata={'input_tokens': 409, 'output_tokens': 4, 'total_tokens': 413, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
